In [2]:
data = spark.read.csv("file:/home/hduser/SparkML-Workouts/insurance.csv", inferSchema = True, header = True)
data.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)



In [3]:
data.describe().show()

+-------+------------------+------+------------------+-----------------+------+---------+------------------+
|summary|               age|   sex|               bmi|         children|smoker|   region|           charges|
+-------+------------------+------+------------------+-----------------+------+---------+------------------+
|  count|              1338|  1338|              1338|             1338|  1338|     1338|              1338|
|   mean| 39.20702541106129|  null|30.663396860986538|  1.0949177877429|  null|     null|13270.422265141257|
| stddev|14.049960379216147|  null| 6.098186911679012|1.205492739781914|  null|     null|12110.011236693992|
|    min|                18|female|             15.96|                0|    no|northeast|         1121.8739|
|    max|                64|  male|             53.13|                5|   yes|southwest|       63770.42801|
+-------+------------------+------+------------------+-----------------+------+---------+------------------+



In [4]:
age = data.corr("age", "charges")
BMI = data.corr("bmi", "charges")

print("Correlation between Age of the person and charges is : {}".format(age))
print("Correlation between BMI of the person and charges is : {}".format(BMI))

Correlation between Age of the person and charges is : 0.299008193330648
Correlation between BMI of the person and charges is : 0.19834096883362903


In [6]:
from pyspark.ml.feature import StringIndexer

category = StringIndexer(inputCol = "sex", outputCol = "gender_categorical")
categorised = category.fit(data).transform(data)
categorised.printSchema()
categorised.show()

root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)
 |-- gender_categorical: double (nullable = false)

+---+------+------+--------+------+---------+-----------+------------------+
|age|   sex|   bmi|children|smoker|   region|    charges|gender_categorical|
+---+------+------+--------+------+---------+-----------+------------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|               1.0|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|               0.0|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|               0.0|
| 33|  male|22.705|       0|    no|northwest|21984.47061|               0.0|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|               0.0|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|        

In [7]:
category = StringIndexer(inputCol = "smoker", outputCol = "smoker_categorical")
categorised = category.fit(categorised).transform(categorised)

category = StringIndexer(inputCol = "region", outputCol = "region_categorical")
categorised = category.fit(categorised).transform(categorised)
categorised.show()

+---+------+------+--------+------+---------+-----------+------------------+------------------+------------------+
|age|   sex|   bmi|children|smoker|   region|    charges|gender_categorical|smoker_categorical|region_categorical|
+---+------+------+--------+------+---------+-----------+------------------+------------------+------------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|               1.0|               1.0|               1.0|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|               0.0|               0.0|               0.0|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|               0.0|               0.0|               0.0|
| 33|  male|22.705|       0|    no|northwest|21984.47061|               0.0|               0.0|               2.0|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|               0.0|               0.0|               2.0|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|               1.0|    

In [12]:
from pyspark.ml.feature import VectorAssembler

concatenating  = VectorAssembler(inputCols=["age","bmi", "children", "gender_categorical", "smoker_categorical", "region_categorical"],outputCol="features")
results = concatenating.transform(categorised)
results.select("features").show(truncate=False)
results.printSchema()


+-----------------------------+
|features                     |
+-----------------------------+
|[19.0,27.9,0.0,1.0,1.0,1.0]  |
|[18.0,33.77,1.0,0.0,0.0,0.0] |
|[28.0,33.0,3.0,0.0,0.0,0.0]  |
|[33.0,22.705,0.0,0.0,0.0,2.0]|
|[32.0,28.88,0.0,0.0,0.0,2.0] |
|[31.0,25.74,0.0,1.0,0.0,0.0] |
|[46.0,33.44,1.0,1.0,0.0,0.0] |
|[37.0,27.74,3.0,1.0,0.0,2.0] |
|[37.0,29.83,2.0,0.0,0.0,3.0] |
|[60.0,25.84,0.0,1.0,0.0,2.0] |
|[25.0,26.22,0.0,0.0,0.0,3.0] |
|[62.0,26.29,0.0,1.0,1.0,0.0] |
|[23.0,34.4,0.0,0.0,0.0,1.0]  |
|[56.0,39.82,0.0,1.0,0.0,0.0] |
|[27.0,42.13,0.0,0.0,1.0,0.0] |
|[19.0,24.6,1.0,0.0,0.0,1.0]  |
|[52.0,30.78,1.0,1.0,0.0,3.0] |
|[23.0,23.845,0.0,0.0,0.0,3.0]|
|[56.0,40.3,0.0,0.0,0.0,1.0]  |
|[30.0,35.3,0.0,0.0,1.0,1.0]  |
+-----------------------------+
only showing top 20 rows

root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: str

In [32]:
for_model = results.select("features", "charges","age","sex")
for_model.show(truncate=False)

+-----------------------------+-----------+---+------+
|features                     |charges    |age|sex   |
+-----------------------------+-----------+---+------+
|[19.0,27.9,0.0,1.0,1.0,1.0]  |16884.924  |19 |female|
|[18.0,33.77,1.0,0.0,0.0,0.0] |1725.5523  |18 |male  |
|[28.0,33.0,3.0,0.0,0.0,0.0]  |4449.462   |28 |male  |
|[33.0,22.705,0.0,0.0,0.0,2.0]|21984.47061|33 |male  |
|[32.0,28.88,0.0,0.0,0.0,2.0] |3866.8552  |32 |male  |
|[31.0,25.74,0.0,1.0,0.0,0.0] |3756.6216  |31 |female|
|[46.0,33.44,1.0,1.0,0.0,0.0] |8240.5896  |46 |female|
|[37.0,27.74,3.0,1.0,0.0,2.0] |7281.5056  |37 |female|
|[37.0,29.83,2.0,0.0,0.0,3.0] |6406.4107  |37 |male  |
|[60.0,25.84,0.0,1.0,0.0,2.0] |28923.13692|60 |female|
|[25.0,26.22,0.0,0.0,0.0,3.0] |2721.3208  |25 |male  |
|[62.0,26.29,0.0,1.0,1.0,0.0] |27808.7251 |62 |female|
|[23.0,34.4,0.0,0.0,0.0,1.0]  |1826.843   |23 |male  |
|[56.0,39.82,0.0,1.0,0.0,0.0] |11090.7178 |56 |female|
|[27.0,42.13,0.0,0.0,1.0,0.0] |39611.7577 |27 |male  |
|[19.0,24.

In [33]:
#Train Test Split:
train_data, test_data = for_model.randomSplit([0.7,0.3])
train_data.describe().show()
print(f"Training Dataset Count:{train_data.count()}")
print(f"Test Dataset Count:{test_data.count()}")
print(f"Full Dataset Count:{for_model.count()}")

+-------+------------------+-----------------+------+
|summary|           charges|              age|   sex|
+-------+------------------+-----------------+------+
|  count|               951|              951|   951|
|   mean|13497.477661050478| 39.1850683491062|  null|
| stddev|12363.225365239392|13.98146999163188|  null|
|    min|         1121.8739|               18|female|
|    max|       63770.42801|               64|  male|
+-------+------------------+-----------------+------+

Training Dataset Count:951
Test Dataset Count:387
Full Dataset Count:1338


In [35]:
#Model Building
from pyspark.ml.regression import LinearRegression

lr_model = LinearRegression(featuresCol= "features",labelCol="charges")
training_model = lr_model.fit(train_data)

In [36]:
#Model Evaluation
output = training_model.evaluate(train_data)
print(output.r2)
print(output.meanSquaredError)
print(output.meanAbsoluteError)

0.7576851541898302
36998718.58164844
4262.777096278992


In [37]:
#Test_data

outputdf = training_model.transform(test_data)
outputdf.show()

+--------------------+----------+---+----+------------------+
|            features|   charges|age| sex|        prediction|
+--------------------+----------+---+----+------------------+
|(6,[0,1],[18.0,33...| 1136.3994| 18|male| 3287.289154854214|
|(6,[0,1],[18.0,34...| 1137.4697| 18|male| 3555.402694022334|
|(6,[0,1],[18.0,41...| 1146.7966| 18|male| 5891.820678201671|
|(6,[0,1],[18.0,53...| 1163.4627| 18|male|10066.731502390976|
|(6,[0,1],[20.0,33...| 1391.5287| 20|male|3681.8081657046714|
|(6,[0,1],[21.0,23...| 1515.3449| 21|male|412.74262902777264|
|(6,[0,1],[21.0,36...| 1534.3045| 21|male| 5162.182465720194|
|(6,[0,1],[22.0,26...| 1664.9996| 22|male|1931.4188632101668|
|(6,[0,1],[23.0,26...| 1815.8759| 23|male| 2071.225467385084|
|(6,[0,1],[29.0,27...| 2867.1196| 29|male| 4097.425051607694|
|(6,[0,1],[34.0,34...| 3935.1799| 34|male| 7554.197332497231|
|(6,[0,1],[40.0,25...| 5415.6612| 40|male| 5903.411236699903|
|(6,[0,1],[48.0,40...| 7804.1605| 48|male|13188.475470966012|
|(6,[0,1

root
 |-- features: vector (nullable = true)
 |-- charges: double (nullable = true)
 |-- prediction: double (nullable = false)

